In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from  sklearn.preprocessing import StandardScaler
from  sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. Dataset Copy number alteration**

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Madonna/CopyNumberAlterationMatrix.csv',index_col=[0]) # here we make first col of data as index col of the data 

#data=pd.read_csv('SomaticPointMutationMatrix.csv',index_col=[0])
#SomaticPointMutationMatrix.csv

In [ ]:
#print(data)
data.shape

(6640, 5478)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
target = 'classLabels'

In [ ]:
data.shape

(6640, 5478)

In [ ]:
X = data.loc[:, data.columns != target]
Y = data.loc[:, data.columns == target]

In [ ]:
X

,A1BG.p,A1BG.n,AACS.p,AACS.n,AAED1.p,AAED1.n,AAMDC.p,AAMDC.n,AARSD1.p,AARSD1.n,AASDH.p,AASDH.n,ABAT.p,ABAT.n,ABCA2.p,ABCA2.n,ABCA7.p,ABCA7.n,ABCB9.p,ABCB9.n,ABCC12.p,ABCC12.n,ABCC9.p,ABCC9.n,ABCD2.p,ABCD2.n,ABCF1.p,ABCF1.n,ABHD12.p,ABHD12.n,ABHD16B.p-DNAJC5.p-TPD52L2.p-UCKL1.p,ABHD16B.n-DNAJC5.n-TPD52L2.n-UCKL1.n,ABO.p,ABO.n,ABTB2.p,ABTB2.n,ACAD8.p,ACAD8.n,ACADS.p-UNC119B.p,ACADS.n,...,ZNF726.n,ZNF727.p,ZNF727.n,ZNF736.p,ZNF736.n,ZNF737.p,ZNF737.n,ZNF738.p,ZNF738.n,ZNF750.p,ZNF750.n,ZNF776.p,ZNF776.n,ZNF8.p,ZNF8.n,ZNF805.p,ZNF805.n,ZNF814.p,ZNF814.n,ZNF883.p,ZNF883.n,ZNF891.p,ZNF891.n,ZNF90.p,ZNF90.n,ZNRF3.p,ZNRF3.n,ZPBP2.p,ZPBP2.n,ZRSR1.p,ZRSR1.n,ZSCAN1.p,ZSCAN1.n,ZSCAN18.p,ZSCAN18.n,ZSCAN22.p,ZSCAN22.n,ZSCAN4.n,ZSWIM7.p,ZSWIM7.n
TCGA.BL.A0C8.01,0,1,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1,0,1
TCGA.BL.A13I.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA.BL.A13J.01,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1
TCGA.BL.A3JM.01,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0
TCGA.BT.A0S7.01,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.YZ.A980.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA.YZ.A982.01,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,1,1,0
TCGA.YZ.A983.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TCGA.YZ.A984.01,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
Y = Y.values.ravel()
Y

array([ 1,  1,  1, ..., 28, 28, 28])

# **2. Splitting of data**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.33, 
                                                    random_state=8)

# **3. Training**

In [ ]:

clf_lr = LogisticRegression(max_iter= 1000)      
lr_baseline_model = clf_lr.fit(x_train,y_train)

In [ ]:
ac = accuracy_score(y_test,lr_baseline_model.predict(x_test))
print(ac)
#from sklearn.metrics import precision_score, recall_score
#precision_score(y_train, y_test, average='micro')


0.7198905109489051


# **4. Feature selection**

In [ ]:
rfecv = RFECV(estimator=clf_lr, step=1000, cv=3, scoring='accuracy')
rfecv = rfecv.fit(x_train, y_train)
print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])

Optimal number of features : 477
Best features : Index(['ABCC12.p',
       'ACR.n-ADM2.n-ALG12.n-ARSA.n-BRD1.n-C22ORF34.n-CHKB.n-CPT1B.n-CRELD2.n-DENND6B.n-HDAC10.n-IL17REL.n-KLHDC7B.n-LMF2.n-MAPK11.n-MAPK12.n-MAPK8IP2.n-MIOX.n-MLC1.n-MOV10L1.n-NCAPH2.n-ODF3B.n-PANX2.n-PIM3.n-PLXNB2.n-RABL2B.n-SBF1.n-SCO2.n-SHANK3.n-SYCE3.n-TRABD.n-TUBGCP6.n-TYMP.n-ZBED4.n',
       'ACTN2.n', 'ADAM32.p', 'ADGB.n', 'ADGRB3.p', 'ADGRB3.n', 'ADSS.p',
       'AGBL4.n', 'AKAP10.n',
       ...
       'ZNF726.p', 'ZNF726.n', 'ZNF736.p', 'ZNF750.n', 'ZNF883.n', 'ZNF891.p',
       'ZNF891.n', 'ZNRF3.n', 'ZRSR1.p', 'ZRSR1.n'],
      dtype='object', length=477)


In [ ]:
rfecv.n_features_
len(rfecv.grid_scores_)
print(len(rfecv.ranking_))

In [ ]:
x_train_rfecv = rfecv.transform(x_train)
x_test_rfecv = rfecv.transform(x_test)

In [ ]:
type(x_train_rfecv)

# **5. Training**

In [ ]:
lr_rfecv_model = clf_lr.fit(x_train_rfecv, y_train)

In [ ]:
ac = accuracy_score(y_test,lr_rfecv_model.predict(x_test_rfecv))
ac

0.7513686131386861

In [ ]:
x_train.columns[rfecv.support_]

Index(['ABCC12.p',
       'ACR.n-ADM2.n-ALG12.n-ARSA.n-BRD1.n-C22ORF34.n-CHKB.n-CPT1B.n-CRELD2.n-DENND6B.n-HDAC10.n-IL17REL.n-KLHDC7B.n-LMF2.n-MAPK11.n-MAPK12.n-MAPK8IP2.n-MIOX.n-MLC1.n-MOV10L1.n-NCAPH2.n-ODF3B.n-PANX2.n-PIM3.n-PLXNB2.n-RABL2B.n-SBF1.n-SCO2.n-SHANK3.n-SYCE3.n-TRABD.n-TUBGCP6.n-TYMP.n-ZBED4.n',
       'ACTN2.n', 'ADAM32.p', 'ADGB.n', 'ADGRB3.p', 'ADGRB3.n', 'ADSS.p',
       'AGBL4.n', 'AKAP10.n',
       ...
       'ZNF726.p', 'ZNF726.n', 'ZNF736.p', 'ZNF750.n', 'ZNF883.n', 'ZNF891.p',
       'ZNF891.n', 'ZNRF3.n', 'ZRSR1.p', 'ZRSR1.n'],
      dtype='object', length=477)

In [ ]:
DF = pd.DataFrame(x_train.columns[rfecv.support_])

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
DF.to_csv('data_copy.csv')
!cp data_copy.csv "drive/My Drive/Madonna"